In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1662


In [2]:
using PProf
using ProfileCanvas
using BenchmarkTools: @btime

import Random
rng = Random.MersenneTwister();

In [3]:
n = 2
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);
#ham = HarmonicOscillator(ω)

l = 10
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = setup_RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);
energy(rhf)

1.17957942722023

In [80]:
ccsd = setup_CCSD(System(rhf))
ccsd = compute_ground_state!(ccsd, max_iter = 1000, tol = 1e-10, verbose=0);
energy(ccsd)

0.8253207495737687

In [87]:
samples = 1e6
metro = Importance(equils = 1e4, samples = samples, step = 0.1);
#metro = Metropolis(equils = 1e4, samples = samples, step = 0.1);
samples

1.0e6

In [62]:
nn = NeuralNetwork([Dense(5), Tanh, Dense(1), Exp], n=n, rng = rng);
slater = Slater(rhf);
wf = SlaterNN(slater, nn);

In [88]:
optimizer = GradientDescent(lr = 0.01, max_iter = 10, tol = 1e-6);
wf, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 8, verbose = true);

E = 0.84 iter = 10/10                                      
No convergence reached, final norm of gradient was 0.001233171816336685


In [38]:
@btime result = blocking(wf, ham, metro, nthreads = 8);
result.E

  127.601 ms (1940 allocations: 6.07 MiB)


1.2124303505496778